In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [4]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [7]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True), X.sum(1), X.sum(0)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]),
 tensor([ 6., 15.]),
 tensor([5., 7., 9.]))

In [8]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制

In [34]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X,X_prob, X_prob.sum(1)

(tensor([[ 0.5763, -0.6178, -0.7373,  1.2925,  1.1702],
         [-1.7378,  0.1741, -0.3021, -0.1804,  0.6029]]),
 tensor([[0.1842, 0.0558, 0.0495, 0.3769, 0.3336],
         [0.0369, 0.2496, 0.1551, 0.1751, 0.3833]]),
 tensor([1., 1.]))

In [35]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

我们创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率， 以及它们对应的标签y。 有了y，我们知道在第一个样本中，第一类是正确的预测； 而在第二个样本中，第三类是正确的预测。 然后使用y作为y_hat中概率的索引， 我们选择第一个样本中第一个类的概率和第二个样本中第三个类的概率。

In [36]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

# 我们只需一行代码就可以实现交叉熵损失函数。因为y为one-hot编码,其他项为0。

In [37]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [ ]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())